<a href="https://colab.research.google.com/github/sakthivelj/tooth_seg/blob/master/nii_gz_to_stl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Install necessary libraries

In [2]:
!pip install nibabel
!pip install numpy
!pip install vtk
!pip install matplotlib
!pip install stl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 MB 7.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for stl: filename=stl-0.0.3-py3-none-any.whl size=5341 sha256=8b51b32fcdea911221de5dc7e32854e3555b054857e80651fdbc52b3407246fb
  Stored in directory: /root/.cache/pip/wheels/e5/ec/d8/f7563096d6e3563c236dd89ee682820474d594cd20bff73deb
Successfully built stl


In [5]:
!pip install numpy-stl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##  Import necessary libraries

In [1]:
import nibabel as nib
import numpy as np
from skimage import measure
import vtk
from vtk.util import numpy_support
import matplotlib.pyplot as plt
from stl import mesh


## Load the .nii.gz file

In [4]:
nii_file = nib.load('/content/drive/MyDrive/cbct/lab/Copy of 1000813648_20180116.nii.gz')
array = nii_file.get_fdata()

# Apply threshold
array[array <= 0.4] = 0
array[array > 0.4] = 1

# Generate surface mesh
verts, faces, _, _ = measure.marching_cubes(array, 0)


# Create the polydata object
polydata = vtk.vtkPolyData()
points = vtk.vtkPoints()
polygons = vtk.vtkCellArray()

# Add the points and polygons to the polydata object
for v in verts:
    points.InsertNextPoint(v)

for f in faces:
    polygon = vtk.vtkPolygon()
    polygon.GetPointIds().SetNumberOfIds(len(f))
    for i in range(len(f)):
        polygon.GetPointIds().SetId(i, f[i])
    polygons.InsertNextCell(polygon)

polydata.SetPoints(points)
polydata.SetPolys(polygons)


# Write STL
stlWriter = vtk.vtkSTLWriter()
stlWriter.SetFileName("output.stl")
stlWriter.SetInputData(polydata)
stlWriter.Write()

1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import files
files.download("output.stl")

In [6]:
!pip install trimesh


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.1/681.1 kB 10.7 MB/s eta 0:00:00


In [7]:
import trimesh

# Load the mesh from file
mesh = trimesh.load_mesh('/content/output.stl')

# This will split the mesh into submeshes, with each being a connected component
submeshes = mesh.split(only_watertight=False)

# Save each submesh as a separate STL file
for i, submesh in enumerate(submeshes):
    submesh.export(f'output_tooth_{i}.stl')

In [8]:
from sklearn.cluster import KMeans
import numpy as np

# assuming that `verts` is your array of vertices
kmeans = KMeans(n_clusters=32, random_state=0).fit(verts)

# create a list to hold the separated teeth
separated_teeth = []

# for each label (tooth), separate the vertices and create a new mesh
for i in range(32):
    tooth_verts = verts[kmeans.labels_ == i]
    tooth_faces = measure.marching_cubes(tooth_verts, 0)[1]
    tooth_mesh = mesh.Mesh(np.zeros(tooth_faces.shape[0], dtype=mesh.Mesh.dtype))
    for j, f in enumerate(tooth_faces):
        for k in range(3):
            tooth_mesh.vectors[j][k] = tooth_verts[f[k],:]
    separated_teeth.append(tooth_mesh)

# now you can save each tooth as a separate STL file
for i, tooth_mesh in enumerate(separated_teeth):
    tooth_mesh.save(f'autput_tooths_{i}.stl')


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


ValueError: ignored